In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/brain-CT

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/brain-CT


In [2]:
import os
import sys
import glob
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as TF
import pandas as pd

from torch.utils.data import Dataset


In [13]:
os.getcwd()

'/content/drive/MyDrive/brain-CT'

In [3]:

%cd ./src
from dataset import CTDataset
from model import *
import utils
%cd ..


/content/drive/MyDrive/brain-CT/src
/content/drive/MyDrive/brain-CT


In [5]:
utils.test()

test


In [ ]:
train = CTDataset("./converted_dataset/train_img","./converted_dataset/train_label")
test = CTDataset("./converted_dataset/val_img","./converted_dataset/val_label")

In [7]:
config = pd.read_json('src/config.json')
resnet152 = Resnet("resnet152")
config = config["resnet152"]
print(config)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


GAMMA                                                                   0.1
base_lr                                                               0.001
batch_size                                                               32
check_point_path                                      my_checkpoint.pth.tar
device                       'cuda' if torch.cuda.is_available() else 'cpu'
epochs                                                                  100
frozen_layer               [conv1, bn1, layer1, layer2, layer3, classifier]
load_model                                                            False
loss_function                                                              
lr_list                                                                  []
lr_steps                                                                 []
model_name                                                        resnet512
num_of_frozen_sub_layer                                                  10
num_workers 

# init

In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
train_loader, val_loader = utils.get_loaders(
        os.path.join(config["train_path"],"image"),
        os.path.join(config["train_path"],"mask"),
        os.path.join(config["validation_path"],"image"),
        os.path.join(config["validation_path"],"mask"),
        config["batch_size"],
        config["num_workers"],
        config["pin_memory"],
    )

if config["load_model"]:
        epoch_losses = np.load('losses\\epoch_losses.npy').tolist()
        val_losses = np.load('.\\losses\\val_losses.npy').tolist()
    else:
        epoch_losses = []
        val_losses = []
losses = []

if config["optimizer"] == "Adam":
    optimizer = optim.Adam(resnet152.getmodel().parameters(), lr=config("base_lr"))

loss_fn = []
if config["loss_function"] == "BCEWithLogitsLoss":
    loss_fn.append(nn.BCEWithLogitsLoss())
if config["loss_function"] == "IoU":
    loss_fn.append(IoULoss())
if config["loss_function"] == "dice":
    loss_fn.append(DiceLoss())
if config["loss_function"] == "MSE":
    loss_fn.append(nn.MSELoss())
#NOTE: temporary loss
loss_fn = nn.MSELoss()
if LOAD_MODEL:
        load_checkpoint(torch.load(config["check_point_path"]), resnet152.get_model())

scaler = torch.cuda.amp.GradScaler()

# training function

In [ ]:
def train_fn(loader, model, optimizer, loss_fn, scaler, losses):
    loop = tqdm(loader)
    for batch_idx, (data, targets) in enumerate(loop):

        data = data
        targets = targets

        # forward
        with torch.cuda.amp.autocast():
            pred = model(data)
            loss, _ = loss_fn(pred, targets)

        # backward
        optimizer.zero_grad()

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # update tqdm loop
        loop.set_postfix(loss=loss.item())
        losses.append(loss.item())

# training loop

In [ ]:
for epoch in range(config["epochs"]):
        print(f"Number of Epoch: {epoch}")
        # print(losses)
        if losses != []:
            epoch_losses.append(np.average(losses))
            losses = []
            np.save('losses\\epoch_losses', epoch_losses)
            print(epoch_losses[-1])

        #TODO: manage loss_combined
        train_fn(train_loader, resnet152.get_model(), optimizer, loss_combined, scaler, losses)

        # save model
        if(epoch % config["save_step"] == 0):
            checkpoint = {
                "state_dict": resnet152.get_model().state_dict(),
                "optimizer": optimizer.state_dict(),
            }
            save_checkpoint(checkpoint)

            # print some examples to a folder
            save_predictions_as_imgs(
                val_loader, resnet152.get_model(), loss_combined, val_losses, folder="./saved_images", device=DEVICE
            )